<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 

Follow instructions step by step until the end and submit your complete notebook as an archive (tar -cf groupXnotebook.tar DL_lab2/).

Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by May 29th 2019 (23:59:59 CET).

# Introduction

In the last Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%.  Can  you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to  more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [27]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from datetime import timedelta
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [6]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288787635
Epoch:  02   =====> Loss= 0.732585251
Epoch:  03   =====> Loss= 0.600357965
Epoch:  04   =====> Loss= 0.536914184
Epoch:  05   =====> Loss= 0.497765308
Epoch:  06   =====> Loss= 0.471163228
Epoch:  07   =====> Loss= 0.451063798
Epoch:  08   =====> Loss= 0.435940319
Epoch:  09   =====> Loss= 0.423547281
Epoch:  10   =====> Loss= 0.413174133
Epoch:  11   =====> Loss= 0.404240680
Epoch:  12   =====> Loss= 0.396970411
Epoch:  13   =====> Loss= 0.390411708
Epoch:  14   =====> Loss= 0.384458124
Epoch:  15   =====> Loss= 0.378928970
Epoch:  16   =====> Loss= 0.374595156
Epoch:  17   =====> Loss= 0.370546356
Epoch:  18   =====> Loss= 0.366674197
Epoch:  19   =====> Loss= 0.362917285
Epoch:  20   =====> Loss= 0.359582534
Epoch:  21   =====> Loss= 0.356729635
Epoch:  22   =====> Loss= 0.353755187
Epoch:  23   =====> Loss= 0.351277490
Epoch:  24   =====> Loss= 0.348786440
Epoch:  25   =====> Loss= 0.346333514
Epoch:  26   =====> Loss= 0.344100199
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png" width="800" height="600" align="center">





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [8]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0., shape=[shape])
  return tf.Variable(initial)

In [9]:
# Definition of CNN parameters

# MNIST images size.
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

# Colour channels (grey-scale).
num_channels = 1

# Number of classes (10 digits).
num_classes = 10

# Layer 1 (Conv 5x5)
filter_size_1 = 5
num_filters_1 = 6
padding_1 = 'SAME'

# Layer 2 (Conv 5x5)
filter_size_2 = 5
num_filters_2 = 16
padding_2 = 'VALID'

# Layer 3
fc_size_1 = 120

# Layer 4
fc_size_2 = 84

# Layer 5
fc_size_3 = num_classes

In [13]:
def LeNet5_Model(input, num_input_channels, filter_size, num_filters, padding, pooling=True):
    
    # Shape of the filter-weights.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Initialize weight and bias given shape.
    weights = weight_variable(shape)
    biases = bias_variable(num_filters)

    # Convolution operation.
    layer = tf.nn.conv2d(input, weights, [1, 1, 1, 1], padding) #Input, filter, strides and padding

    # Add the biases to the results of the convolution.
    layer += biases

    # Use pooling according to LeNet-5 architecture.
    if pooling == True:
        # This is 2x2 max-pooling, which applies in all cases of CNN architecture.
        layer = tf.nn.max_pool(layer, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID') # value, ksize, strides and padding

    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)

    return layer, weights

In [14]:
def flatten_layer(layer):
    
    # Shape of the input layer.
    layer_shape = layer.get_shape()

    # Number of features
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features

In [15]:
def fc_layer(input, num_inputs, num_outputs):

    # Create new weights and biases.
    weights = weight_variable(shape=[num_inputs, num_outputs])
    biases = bias_variable(shape=num_outputs)

    # Calculate the layer as the matrix multiplication of the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases
    
    # Passing ReLu for non linear operation.
    layer = tf.nn.relu(layer)

    return layer

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

<div class="alert alert-info">

<b> Layer C1</b>. 
    Parameters = Weight + Bias = 5x5x1x6 + 6 = 156.
    Connections = 28x28x156 = 122304.
    
<b> Layer S2</b>. 
    Parameters = (Coefficient + Bias) x Filters = (1+1) x 6 = 12.
    Connections = 14x14x30 = 5880.

<b> Layer C3</b>. 
    Parameters = Weight + Bias = 5x5x6x10 + 16 = 1516.
    Connections = 10x10x1516 = 151600.
    
<b> Layer S4</b>. 
    Parameters = (Coefficient + Bias) x Filters = (1+1) x 16 = 32.
    Connections = 5x5x80 = 2000
    
<b> Layer FC5</b>. 
    Parameters =  Weight + Bias = (NodesinS4 x Outputs) + Bias = (5x5x16 x 120) + 120 = 48120.

<b> Layer FC6</b>. 
    Parameters =  Weight + Bias = (NodesinFC5 x Outputs) + Bias = (120 x 84) + 84 = 10164.
    
</div>

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [8]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'

# Model, loss function and accuracy

x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='images')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

# Layer 1: Convolution
layer_conv1, weights_conv1 = LeNet5_Model(x_image, num_channels, filter_size_1, num_filters_1, padding_1, pooling=True)

# Layer 2: Convolution
layer_conv2, weights_conv2 = LeNet5_Model(layer_conv1, num_filters_1, filter_size_2, num_filters_2, padding_2, pooling=True)

# Flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

# Layer 3: Fully Connected
fc_layer_1 = fc_layer(layer_flat, num_features, fc_size_1)

# Layer 4: Fully Connected
fc_layer_2 = fc_layer(fc_layer_1, fc_size_1, fc_size_2)

# Layer 5: Fully Connected
fc_layer_3 = fc_layer(fc_layer_2, fc_size_2, fc_size_3)

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [19]:
def evaluate(logits, labels):
    # logits will be the outputs of your model, labels will be one-hot vectors corresponding to the actual labels
    # logits and labels are numpy arrays
    # this function should return the accuracy of your model
    
    # Measuring prediction between model and real data
    difference_model_pred = tf.equal(logits, labels)
    
    # Return accurancy
    return tf.reduce_mean(tf.cast(difference_model_pred, tf.float32))

In [33]:
# Prediction label from model (max value of Softmax operation).
y_pred_cls = tf.argmax(tf.nn.softmax(fc_layer_3), axis=1)

# Real label from data
y_true_cls = tf.argmax(y_true, axis=1)

# Evaluation of the accuracy
accuracy = evaluate(y_pred_cls, y_true_cls)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [34]:
# Import packages
import time
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_LeNet-5_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_LeNet-5_SGD", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()
# Split the test-set into smaller batches of this size.
test_batch_size = 256

In [35]:
# Implementation of an optimizer

# Loss function.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_layer_3,
                                                        labels=y_true)
# Loss function.
cost = tf.reduce_mean(cross_entropy)

# Accuracy.
accuracy = evaluate(y_pred_cls, y_true_cls)

# Gradient Descent optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [24]:
# Initializing the variables
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    # optimizer and cost are the same kinds of objects as in Section 1
    # Train your model
    
    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    for epoch in range(n_epochs):
        
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
               
            # Batch data for train_x and train_y
            x_batch, y_true_batch = mnist.train.next_batch(batch_size, 
                                                           shuffle=(i==0))
            # Feed train batch data as dic.
            feed_train = {x: x_batch, 
                          y_true: y_true_batch}

            # Run graph.
            _, c, _, summary = sess.run([optimizer, cost, accuracy, merged_summary_op],
                                          feed_dict=feed_train)
            
            # Compute average loss
            avg_cost += c / total_batch
            
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            acur = sess.run(accuracy, feed_dict=feed_train)
            print("Epoch: ", '%02d' % (epoch+1), " ===> Loss=", "{:.5f}".format(avg_cost), ", Training acurracy=", "{:.2%}".format(acur))
   
    # Ending time.
    end_time = time.time()
    
    # Difference between start and end-times.
    dif = end_time - start_time

    # Print the time-usage.
    print('\nTime usage: ' + str(timedelta(seconds=int(round(dif)))))
    
    pass

In [25]:
def test_accuracy():

    # Number of images in the test-set.
    num_test = len(mnist.test.images)

    # Allocate an array for the predicted classes which
    # will be calculated in batches and filled into this array.
    cls_pred = np.zeros(shape=num_test, dtype=np.int)

    i = 0

    while i < num_test:
        # The ending index for the next batch is denoted j.
        j = min(i + test_batch_size, num_test)

        # Get the images from the test-set between index i and j.
        images = mnist.test.images[i:j, :]

        # Get the associated labels.
        labels = mnist.test.labels[i:j, :]

        # Create a feed-dict with these images and labels.
        feed_dict = {x: images,
                     y_true: labels}

        # Calculate the predicted class using TensorFlow.
        cls_pred[i:j] = sess.run(y_pred_cls, feed_dict=feed_dict)

        # Set the start-index for the next batch to the
        # end-index of the current batch.
        i = j

    # Convenience variable for the true class-numbers of the test-set.
    cls_true = np.argmax(mnist.test.labels, axis=1)

    # Create a boolean array whether each image is correctly classified.
    correct = (cls_true == cls_pred)

    # Calculate the number of correctly classified images.
    # When summing a boolean array, False means 0 and True means 1.
    correct_sum = correct.sum()

    # Classification accuracy is the number of correctly classified
    # images divided by the total number of images in the test-set.
    acc = float(correct_sum) / num_test

    print('\n\nAccuracy Test Set: {0:.1%} ({1} / {2})'.format(acc, correct_sum, num_test))
    pass

In [38]:
# Print the accuracy on testing data
with tf.Session() as sess:
    sess.run(init)
    # Write logs to Tensorboard.
    summary_writer = tf.summary.FileWriter('lab_2/log_files', graph=tf.get_default_graph())
    # Run train process.
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    print('\nOptimization Finished!')
    summary_writer.flush()
    # Print test accurancy.
    test_accuracy()

Epoch:  01  ===> Loss= 1.71015 , Training acurracy= 78.12%
Epoch:  02  ===> Loss= 0.62713 , Training acurracy= 81.25%
Epoch:  03  ===> Loss= 0.47038 , Training acurracy= 92.19%
Epoch:  04  ===> Loss= 0.22808 , Training acurracy= 96.09%
Epoch:  05  ===> Loss= 0.18567 , Training acurracy= 94.53%
Epoch:  06  ===> Loss= 0.16172 , Training acurracy= 96.09%
Epoch:  07  ===> Loss= 0.14352 , Training acurracy= 96.09%
Epoch:  08  ===> Loss= 0.12959 , Training acurracy= 96.09%
Epoch:  09  ===> Loss= 0.11882 , Training acurracy= 96.88%
Epoch:  10  ===> Loss= 0.10994 , Training acurracy= 96.88%
Epoch:  11  ===> Loss= 0.10137 , Training acurracy= 97.66%
Epoch:  12  ===> Loss= 0.09537 , Training acurracy= 100.00%
Epoch:  13  ===> Loss= 0.08983 , Training acurracy= 99.22%
Epoch:  14  ===> Loss= 0.08466 , Training acurracy= 98.44%
Epoch:  15  ===> Loss= 0.08056 , Training acurracy= 98.44%
Epoch:  16  ===> Loss= 0.07681 , Training acurracy= 100.00%
Epoch:  17  ===> Loss= 0.07321 , Training acurracy= 99

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Please put your loss and accuracy curves here.

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         0.9837        |        0.9789          |       
| Training Time        |         09:20        |        11:14          |  

- Which optimizer gives the best accuracy on test data?

According to the results, in both methods we can see similar results. Even if we can see that Gradient Descent is faster than AdamOptimizer, in terms of accuracy, both show similar results. 

In our opinion it is difficult to compare the accuracies because they start randomly at differents points and also because AdamOptimizer includes momentum in its algorithm to speed up calculation.


In [39]:
tf.reset_default_graph()

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='images')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

# Layer 1: Convolution
layer_conv1, weights_conv1 = LeNet5_Model(x_image, num_channels, filter_size_1, num_filters_1, padding_1, pooling=True)

# Layer 2: Convolution
layer_conv2, weights_conv2 = LeNet5_Model(layer_conv1, num_filters_1, filter_size_2, num_filters_2, padding_2, pooling=True)

# Flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

# Layer 3: Fully Connected
fc_layer_1 = fc_layer(layer_flat, num_features, fc_size_1)

# Layer 4: Fully Connected
fc_layer_2 = fc_layer(fc_layer_1, fc_size_1, fc_size_2)

# Layer 5: Fully Connected
fc_layer_3 = fc_layer(fc_layer_2, fc_size_2, fc_size_3)

In [40]:
# Prediction label from model (max value of Softmax operation).
y_pred_cls = tf.argmax(tf.nn.softmax(fc_layer_3), axis=1)

# Real label from data
y_true_cls = tf.argmax(y_true, axis=1)

In [41]:
accuracy = evaluate(y_pred_cls, y_true_cls)

In [42]:
# Loss function.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_layer_3,
                                                        labels=y_true)
# cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Loss_Adam'):
    cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Accuracy_Adam'):
    accuracy = evaluate(y_pred_cls, y_true_cls)

# Gradient Descent optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar('Cost_Adam', cost)

# Create a summary to monitor accuracy tensor
tf.summary.scalar('Accuracy_Adam', accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [44]:
with tf.Session() as sess:    
    sess.run(init)
    # Write logs to Tensorboard.
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Run train process.
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    print('\nOptimization Finished!')
    summary_writer.flush()
    # Print test accurancy.
    test_accuracy()

Epoch:  01  ===> Loss= 0.79535 , Training acurracy= 76.56%
Epoch:  02  ===> Loss= 0.41488 , Training acurracy= 87.50%
Epoch:  03  ===> Loss= 0.28971 , Training acurracy= 85.16%
Epoch:  04  ===> Loss= 0.27597 , Training acurracy= 90.62%
Epoch:  05  ===> Loss= 0.27145 , Training acurracy= 91.41%
Epoch:  06  ===> Loss= 0.27012 , Training acurracy= 88.28%
Epoch:  07  ===> Loss= 0.26911 , Training acurracy= 91.41%
Epoch:  08  ===> Loss= 0.27423 , Training acurracy= 89.06%
Epoch:  09  ===> Loss= 0.26597 , Training acurracy= 89.84%
Epoch:  10  ===> Loss= 0.16068 , Training acurracy= 99.22%
Epoch:  11  ===> Loss= 0.05407 , Training acurracy= 99.22%
Epoch:  12  ===> Loss= 0.04410 , Training acurracy= 99.22%
Epoch:  13  ===> Loss= 0.03722 , Training acurracy= 99.22%
Epoch:  14  ===> Loss= 0.04582 , Training acurracy= 97.66%
Epoch:  15  ===> Loss= 0.04092 , Training acurracy= 100.00%
Epoch:  16  ===> Loss= 0.03624 , Training acurracy= 100.00%
Epoch:  17  ===> Loss= 0.03252 , Training acurracy= 99

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [3]:
def LeNet5_Model_Dropout(input, num_input_channels, filter_size, num_filters, padding, pooling=True):

    # Shape of the filter-weights.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Initialize weight and bias given shape.
    weights = weight_variable(shape)
    biases = bias_variable(num_filters)

    # Convolution operation.
    layer = tf.nn.conv2d(input, weights, [1, 1, 1, 1], padding) #Input, filter, strides and padding

    # Add the biases to the results of the convolution.
    layer += biases

    # Use pooling according to LeNet-5 architecture.
    if pooling == True:
        # This is 2x2 max-pooling, which applies in all cases of CNN architecture.
        layer = tf.nn.max_pool(layer, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID') # value, ksize, strides and padding

    # Layer dropout with keep_prob = 0.75
    layer = tf.nn.dropout(layer, keep_prob = 0.75)
    # Rectified Linear Unit (ReLU).
    layer = tf.nn.relu(layer)

    return layer, weights

In [16]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
logs_path = 'log_files/'

# Model, loss function and accuracy

x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='images')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

# Layer 1: Convolution
layer_conv1, weights_conv1 = LeNet5_Model(x_image, num_channels, filter_size_1, num_filters_1, padding_1, pooling=True)

# Layer 2: Convolution
layer_conv2, weights_conv2 = LeNet5_Model(layer_conv1, num_filters_1, filter_size_2, num_filters_2, padding_2, pooling=True)

# Flatten layer
layer_flat, num_features = flatten_layer(layer_conv2)

# Layer 3: Fully Connected
fc_layer_1 = fc_layer(layer_flat, num_features, fc_size_1)

# Layer 4: Fully Connected
fc_layer_2 = fc_layer(fc_layer_1, fc_size_1, fc_size_2)

# Layer 5: Fully Connected
fc_layer_3 = fc_layer(fc_layer_2, fc_size_2, fc_size_3)

In [17]:
# Prediction label from model (max value of Softmax operation).
y_pred_cls = tf.argmax(tf.nn.softmax(fc_layer_3), axis=1)

# Real label from data
y_true_cls = tf.argmax(y_true, axis=1)

In [20]:
accuracy = evaluate(y_pred_cls, y_true_cls)

In [21]:
# Loss function.
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc_layer_3, labels=y_true)

with tf.name_scope('Loss_Drop'):
    cost = tf.reduce_mean(cross_entropy)

with tf.name_scope('Accuracy_Drop'):
    accuracy = evaluate(y_pred_cls, y_true_cls)

# Gradient Descent optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [30]:
# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar('Cost_Drop', cost)

# Create a summary to monitor accuracy tensor
tf.summary.scalar('Accuracy_Drop', accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Initializing the variables
no_iter = 0

# Split the test-set into smaller batches of this size.
test_batch_size = 256

In [35]:
with tf.Session() as sess:
    sess.run(init)
    # Write logs to Tensorboard.
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Run train process.
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)
    print('\nOptimization Finished!')
    summary_writer.flush()
    # Print test accurancy.
    test_accuracy()

Epoch:  01  ===> Loss= 1.69865 , Training acurracy= 85.16%
Epoch:  02  ===> Loss= 0.47646 , Training acurracy= 90.62%
Epoch:  03  ===> Loss= 0.26558 , Training acurracy= 93.75%
Epoch:  04  ===> Loss= 0.20104 , Training acurracy= 95.31%
Epoch:  05  ===> Loss= 0.16502 , Training acurracy= 95.31%
Epoch:  06  ===> Loss= 0.14133 , Training acurracy= 96.88%
Epoch:  07  ===> Loss= 0.12429 , Training acurracy= 97.66%
Epoch:  08  ===> Loss= 0.11291 , Training acurracy= 95.31%
Epoch:  09  ===> Loss= 0.10379 , Training acurracy= 96.09%
Epoch:  10  ===> Loss= 0.09526 , Training acurracy= 97.66%
Epoch:  11  ===> Loss= 0.09015 , Training acurracy= 99.22%
Epoch:  12  ===> Loss= 0.08474 , Training acurracy= 97.66%
Epoch:  13  ===> Loss= 0.07965 , Training acurracy= 100.00%
Epoch:  14  ===> Loss= 0.07646 , Training acurracy= 100.00%
Epoch:  15  ===> Loss= 0.07327 , Training acurracy= 99.22%
Epoch:  16  ===> Loss= 0.06920 , Training acurracy= 97.66%
Epoch:  17  ===> Loss= 0.06688 , Training acurracy= 97